In [23]:
import pandas as pd
import statistics

processed_data_path = "C:/Users/anush/OneDrive/Documents/Sem3/Explorative Information Visualization/Project/Data/CleanedData"

pizza_ingredients = ["wheat", "tomato", "onion", "peppers", "vegetables", "fruits"]
country_list = ["India", "United States", "Egypt"]
year_min = 1979
year_max = 2019 #new max year is 2019 (not 2023)

In [2]:
foodLoss = pd.read_csv(processed_data_path + "/country-food-year-loss.csv")

In [3]:
foodLoss

,country,commodity,year,loss_percentage
0,India,Wheat,2016,7.04
1,India,Wheat,2016,0.86
2,India,Wheat,2016,3.36
3,India,Wheat,2016,1.43
4,India,Wheat,2016,4.07
...,...,...,...,...
655,United States of America,"Chillies and peppers, green (<i>Capsicum</i> s...",1979,8.00
656,United States of America,Tomatoes,1979,15.00
657,United States of America,"Other beans, green",1979,6.00
658,United States of America,"Onions and shallots, green",1979,6.00


In [6]:
set(foodLoss["commodity"])

{'Cantaloupes and other melons',
 'Chillies and peppers, dry (<i>Capsicum</i> spp., <i>Pimenta</i> spp.), raw',
 'Chillies and peppers, green (<i>Capsicum</i> spp. and <i>Pimenta</i> spp.)',
 'Onions and shallots, green',
 'Other beans, green',
 'Other citrus fruit, n.e.c.',
 'Other fruits, n.e.c.',
 'Other pome fruits',
 'Other pulses n.e.c.',
 'Other stimulant, spice and aromatic crops, n.e.c.',
 'Other tropical and subtropical fruits, n.e.c.',
 'Other vegetables, fresh n.e.c.',
 'Tomatoes',
 'Wheat'}

In [10]:
'''
The dataframe new_foodLoss contains percentages of crop lost in different regions of the country.
So we are adding them up in this piece of code.
Some errors may have occurred, so some sums are more than 100. Will remove those entries and interpolate them.
'''

new_foodLoss = {}
#foodLoss_country = foodLoss[foodLoss["country"] == "India"]
for idx, row in foodLoss.iterrows():
    ingredient = ""
    found = 0
    for ing in pizza_ingredients:
        if ing in row["commodity"].lower():
            ingredient = ing
            found = 1
            break
    if found == 0:
        continue
    else:
        keyname = "" + row["country"] + "_" + ingredient + "_" + str(row["year"])
        new_foodLoss[keyname] = new_foodLoss.get(keyname, 0) + row["loss_percentage"]            

In [11]:
new_foodLoss

{'India_wheat_2016': 24.189999999999998,
 'India_tomato_2016': 28.04,
 'India_onion_2016': 37.239999999999995,
 'India_fruits_2016': 33.96,
 'India_peppers_2016': 8.110000000000001,
 'India_wheat_2015': 15.3,
 'India_tomato_2015': 35.0,
 'India_wheat_2014': 9.85,
 'India_tomato_2014': 24.88,
 'India_onion_2014': 16.39,
 'India_fruits_2014': 19.45,
 'India_peppers_2014': 12.99,
 'India_wheat_2013': 34.42,
 'India_tomato_2013': 12.44,
 'India_onion_2012': 119.91,
 'India_wheat_2012': 12.93,
 'India_tomato_2010': 78.74999999999999,
 'India_onion_2010': 27.5,
 'India_fruits_2010': 68.85,
 'India_tomato_2009': 139.17000000000002,
 'India_vegetables_2009': 10.41,
 'India_peppers_2009': 41.67000000000001,
 'India_tomato_2008': 35.0,
 'India_onion_2006': 27.750000000000004,
 'India_wheat_2006': 9.88,
 'India_tomato_2006': 23.06,
 'India_fruits_2006': 8.62,
 'India_peppers_2006': 10.19,
 'India_tomato_2005': 16.65,
 'India_onion_2004': 15.7,
 'India_tomato_2003': 20.0,
 'India_tomato_2002': 34.

In [14]:
'''
Converting dict into dataframe with different columns
Removing entries that have percentage more than 100.
'''
country_list = []
commodity_list = []
year_list = []
loss_list = []

for key in new_foodLoss.keys():
    if new_foodLoss[key] < 90:
        splitlist = key.split("_")
        if splitlist[0] == "United States of America":
            country_list.append("USA")
        else:
            country_list.append(splitlist[0])
        commodity_list.append(splitlist[1])       
        year_list.append(splitlist[2])
        loss_list.append(new_foodLoss[key])

new_dict = {"country": country_list, "commodity": commodity_list, "year": year_list, "loss_percentage": loss_list}
foodLoss_2 = pd.DataFrame(new_dict)

In [15]:
foodLoss_2

,country,commodity,year,loss_percentage
0,India,wheat,2016,24.19
1,India,tomato,2016,28.04
2,India,onion,2016,37.24
3,India,fruits,2016,33.96
4,India,peppers,2016,8.11
...,...,...,...,...
201,USA,vegetables,1980,7.00
202,USA,peppers,1979,8.00
203,USA,tomato,1979,15.00
204,USA,onion,1979,6.00


In [16]:
#foodLoss_2.to_csv(processed_data_path + "/country-food-year-totalLoss.csv", index=False)

In [28]:
'''
To be able to see what values are missing, I will convert foodLoss_2 to a different format:
Columns are as follows: year columns will contain the loss percentage.
country commodity 1979 1980 1981 .... 2023
'''

pivot_df = foodLoss_2.pivot(index=['country', 'commodity'], columns='year', values='loss_percentage')

# Reindex to include all years from 1979 to 2023
#all_years = list(range(1979, 2024))
#pivot_df = pivot_df.reindex(columns=all_years)

# Reset index for readability (optional)
pivot_df = pivot_df.reset_index()

In [29]:
pivot_df.columns

Index(['country', 'commodity', '1979', '1980', '1981', '1982', '1983', '1984',
       '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993',
       '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002',
       '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019'],
      dtype='object', name='year')

In [30]:
pivot_df

year,country,commodity,1979,1980,1981,1982,1983,1984,1985,1986,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Egypt,fruits,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,23.0,NaN,NaN,NaN,NaN
1,Egypt,tomato,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,43.00,NaN,15.0,NaN,NaN,NaN,NaN
2,Egypt,wheat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,India,fruits,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,68.85,NaN,NaN,NaN,19.45,NaN,33.96,NaN,NaN,NaN
4,India,onion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27.50,NaN,NaN,NaN,16.39,NaN,37.24,NaN,NaN,NaN
5,India,peppers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,12.99,NaN,8.11,NaN,NaN,NaN
6,India,tomato,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,78.75,NaN,NaN,12.44,24.88,35.0,28.04,NaN,NaN,NaN
7,India,vegetables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,India,wheat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,12.93,34.42,9.85,15.3,24.19,NaN,NaN,NaN
9,USA,fruits,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,21.30,NaN,NaN,NaN,NaN,NaN


In [31]:
#pivot_df.to_csv(processed_data_path + "/country-commodity-yearwise-timeseries-data.csv", index=False)

In [36]:
foodLoss_interpolated = pd.read_csv(processed_data_path + "/country-commodity-yearwise-timeseries-data-interpolatedData.csv")

In [37]:
foodLoss_interpolated

,commodity,country,1979,1980,1981,1982,1983,1984,1985,1986,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,onion,Egypt,20.00,20.00,20.00,20.00,20.00,20.00,20.00,20.00,...,20.00,20.0000,20.000,20.0000,20.00,23.000,23.00,23.00,23.00,23.00
1,peppers,Egypt,20.00,20.00,20.00,20.00,20.00,20.00,20.00,20.00,...,20.00,20.0000,20.000,20.0000,20.00,23.000,23.00,23.00,23.00,23.00
2,tomato,Egypt,20.00,20.00,20.00,20.00,20.00,20.00,43.00,43.00,...,43.00,43.0000,43.000,43.0000,29.00,15.000,15.00,15.00,15.00,15.00
3,wheat,Egypt,20.00,20.00,20.00,20.00,20.00,20.00,7.20,7.20,...,14.00,14.0000,14.000,14.0000,14.00,14.000,14.00,14.00,14.00,14.00
4,onion,India,30.00,30.00,30.00,30.00,30.00,30.00,30.00,30.00,...,27.50,24.7225,21.945,19.1675,16.39,26.815,37.24,37.24,37.24,37.24
5,peppers,India,10.19,10.19,10.19,10.19,10.19,10.19,10.19,10.19,...,41.67,34.5000,27.330,20.1600,12.99,10.550,8.11,8.11,8.11,8.11
6,tomato,India,NaN,13.00,13.00,13.00,13.00,13.00,13.00,13.00,...,78.75,56.6400,34.540,12.4400,24.88,35.000,28.04,28.04,28.04,28.04
7,wheat,India,4.95,4.95,4.95,4.95,4.95,4.95,4.95,4.95,...,10.41,10.4100,12.930,34.4200,9.85,15.300,24.19,24.19,24.19,24.19
8,onion,USA,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,...,6.00,6.0000,6.000,6.0000,6.00,6.000,6.00,6.00,6.00,6.00
9,peppers,USA,8.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00,...,8.00,8.0000,8.000,8.0000,8.00,8.000,8.00,8.00,8.00,8.00
